# Load the model

In [57]:
import sklearn.externals as extjoblib
import joblib

# Upload the saved svm model:
model = joblib.load('LinearSVC.npy')
print(model)

LinearSVC()


In [61]:

from skimage import color
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
from skimage import exposure


# define the sliding window:
def sliding_window(image, stepSize, windowSize):# image is the input, step size is the no.of pixels needed to skip and windowSize is the size of the actual window
    # slide a window across the image
    for y in range(0, image.shape[0], stepSize):# this line and the line below actually defines the sliding part and loops over the x and y coordinates
        for x in range(0, image.shape[1], stepSize):
            # yield the current window
            yield (x, y, image[y: y + windowSize[1], x:x + windowSize[0]])
